7.8
* change hard code column index to column names (change matrix to dataframe, change index to column name) [DONE]
* use sklearn to do the splitting [DONE]
* calculate and print out proportion (1. # of data in a node / total #, 2. proportion of trt and ctl) [DONE]
* change the output into a sklearn tree structure [CAN'T FIND]
* Incorporate the global variable [DONE]
* implement prediction function [DONE]

7.9:
* plot the tree somehow [in progress]
    * rpart.plot(x) uses a similar way to store the tree structure. Maybe find the rpart code and translate into python
* implement random forest [DONE]

In [1]:
import numpy as np
from scipy import stats
import pandas as pd
import random
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

# Functions to build tree

In [2]:
def causal_train_test_split(data, predictors, response, treatment, test_size = 0.25, estimation_size = 0.33):
    
    #define PROP as a global variable to use later
    global PROP
    PROP = 1 - estimation_size
    
    #split the whole set into training set and test set
    train_set, test_set = train_test_split(data, test_size=test_size)
    #split the training set into training sample and estimation sample
    training_sample, estimation_sample = train_test_split(train_set, test_size=estimation_size)
    
    #randomly assign labels to training sample and estimation sample
    training_sample.insert(loc = 0, column = 'TRAIN_ESTIMATION_IND', value = np.ones(len(training_sample)))
    estimation_sample.insert(loc = 0, column = 'TRAIN_ESTIMATION_IND', value = np.zeros(len(estimation_sample)))
    new_train_set = pd.concat([training_sample, estimation_sample])
    
    #take the subset of the original dataset with necessary columns
    new_train_set = new_train_set[['TRAIN_ESTIMATION_IND'] + predictors + treatment + response]
    test_set = test_set[predictors + treatment + response]
    
    return new_train_set, test_set

In [3]:
def data_split(index, value, dataset):
    """ 
    A function seperate a dataset into two numpy matrices 
    given the index of an attribute and a split value for that attribute
    
    Input:
    ------
        index(int): the index of the columns of the dataset
        value(float): the value to be compared with
        dataset(numpy array): the dataset to split
    
    Output:
    ------:
        left(numpy array): the dataset that is split(left half)
        right(numpy array): the dataset that is split(right half)
    
    """
    #get the number of columns of the matrix
    dim = dataset.shape[1]
    
    left, right = np.empty(shape=[0, dim]), np.empty(shape=[0, dim])
    
    for row in dataset:
        if row[index] < value:
            left = np.append(left, [row], axis = 0)
        else:
            right = np.append(right, [row], axis = 0)
    return left, right

In [4]:
def get_emse(train, est, row, index):
    
    # check the cardinality of the training and estimation samples, if size < *threshold*
    # then can not do the calculation
    train_size = len(train)
    est_size = len(est)
    
    # split both training sample and estimation sample using the same rule
    left_train, right_train = data_split(index, row[index], train)
    left_est, right_est = data_split(index, row[index], est)
    

    ### Calculate the estimated treatment effect
            
    # get the cardinality of training sample for both leaves
    left_train_size = len(left_train)
    right_train_size = len(right_train)
    
    # calculate the treatment effect for both leaves, 
    left_est_response_trt = get_response(left_est, 'treatment') 
    left_est_response_ctl = get_response(left_est, 'control') 
    right_est_response_trt = get_response(right_est, 'treatment') 
    right_est_response_ctl = get_response(right_est, 'control') 
    #check cardinality of each leaf, make sure each leaf has at least *threshold* (chould be changed by user)
    # treatment and n control to do the calculation
    
    left_trt_effect = left_est_response_trt.mean() - left_est_response_ctl.mean()
    right_trt_effect = right_est_response_trt.mean() - right_est_response_ctl.mean()
    
    # then calculated the estimated squared treatment effect
    e_trt_effect = (left_train_size * (left_trt_effect ** 2) + right_train_size * (right_trt_effect ** 2))/(train_size)
    
            
    ### Calculate the estimated variance
    left_var = np.var(left_est_response_trt) / PROP + np.var(left_est_response_ctl) / (1 - PROP)
    right_var = np.var(right_est_response_trt) / PROP + np.var(right_est_response_ctl) / (1 - PROP)
    e_var = (1 / train_size + 1 / est_size) * (left_var + right_var)
    
    
    ### Calculate EMSE
    emse = e_trt_effect - e_var    
    
    return emse
    
def get_split_emse(dataset):
 
    train = dataset[dataset[:,0] == 1]
    est = dataset[dataset[:,0] == 0]


    # initialize values to return
    b_index, b_value, b_score, b_groups = float('inf'), float('inf'), float('-inf'), None
    
    for index in range(1, train.shape[1] - 2):
        for row in train:
            groups = data_split(index, row[index], dataset)
            emse = get_emse(train, est, row, index)
            # if mse score gets improved (reduced actually), update the information
            if emse > b_score:# and emse is not np.nan:
                b_index, b_value, b_score, b_groups = index, row[index], emse, groups   
                
    ret_dict =  {'index':b_index, 'value':b_value, 'groups':b_groups}
    return ret_dict

In [5]:
def get_response(dataset, trt):
    if trt == 'treatment':
        return dataset[dataset[:,-2] == 1][:,-1]
    elif trt == 'control':
        return dataset[dataset[:,-2] == 0][:,-1]

In [6]:
# get the split based on criterion
def get_split(dataset, criterion):
    """ 
    A function to split the data based on splitting criterion specified by user
    
    Input:
    ------
        dataset(np array): a dataset in the form of a numpy matrix
        criterion(str): a str to indicate the criterion specified by user
    
    Output:
    ------:
        the same output as functions get_split_xxx
    
    """    
    if criterion == 'mse':
        return get_split_mse(dataset)
    if criterion == 'causal':
        return get_split_emse(dataset)    
    elif criterion == 'gini':
        return get_split_gini(dataset)

In [53]:
# Create a terminal node value
def to_terminal_gini(group):
    response = group[:,-1]
    return stats.mode(response)[0][0] # this could be optimized

def to_terminal_mse(group):
    response = group[:,-1]
    return np.mean(response)

def to_terminal_emse(group):
    est_trt = get_response(group, 'treatment')
    est_ctl = get_response(group, 'control')
    
    causal_effect = np.mean(est_trt) - np.mean(est_ctl)
    proportion_of_data = (len(est_trt) + len(est_ctl)) / TOTAL_DATA_COUNT
    
    return round(causal_effect, 3), round(proportion_of_data * 100, 1)
    
    
def to_terminal(group, criterion):
    if criterion == 'gini':
        return to_terminal_gini(group)
    elif criterion == 'mse':
        return to_terminal_mse(group)
    elif criterion == 'causal':
        return to_terminal_emse(group)

In [8]:
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth, criterion):
    
    left, right = node['groups']
    
    left_train = left[left[:,0] == 1]
    left_est = left[left[:,0] == 0]
    right_train = right[right[:,0] == 1]
    right_est = right[right[:,0] == 0]    
    
    left_train_response_trt = get_response(left_train, 'treatment')
    left_train_response_ctl = get_response(left_train, 'control')
    right_train_response_trt = get_response(right_train, 'treatment')
    right_train_response_ctl = get_response(right_train, 'control')  
    
    left_est_response_trt = get_response(left_est, 'treatment')
    left_est_response_ctl = get_response(left_est, 'control')
    right_est_response_trt = get_response(right_est, 'treatment')
    right_est_response_ctl = get_response(right_est, 'control')
    
    del(node['groups'])
    
    if len(left) == 0 or len(right) == 0:
        node['left'] = node['right'] = to_terminal(np.append(left, right, axis = 0), criterion)
        return
    
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left, criterion), to_terminal(right, criterion)
        return
    
    # process left child
    if (len(left) <= min_size or len(left_est_response_trt) <= min_size or len(left_est_response_ctl) <= min_size or
        len(right_est_response_trt) <= min_size or len(right_est_response_ctl) <= min_size or 
        len(left_train_response_trt) <= min_size or len(left_train_response_ctl) <= min_size or 
        len(right_train_response_trt) <= min_size or len(right_train_response_ctl) <= min_size):
        node['left'] = to_terminal(left, criterion)
    else:
        node['left'] = get_split(left, criterion)
        if node['left']['groups'] is None:
            node['left'] = to_terminal(left, criterion)
        else:
            split(node['left'], max_depth, min_size, depth+1, criterion)
        
    # process right child
    if (len(right) <= min_size or len(left_est_response_trt) <= min_size or len(left_est_response_ctl) <= min_size or
        len(right_est_response_trt) <= min_size or len(right_est_response_ctl) <= min_size or 
        len(left_train_response_trt) <= min_size or len(left_train_response_ctl) <= min_size or 
        len(right_train_response_trt) <= min_size or len(right_train_response_ctl) <= min_size):
        node['right'] = to_terminal(right, criterion)
    else:
        node['right'] = get_split(right, criterion)
        if node['right']['groups'] is None:
            node['right'] = to_terminal(right, criterion)
        else:
            split(node['right'], max_depth, min_size, depth+1, criterion)

In [9]:
# Build a decision tree
def causalTree(train, max_depth, min_size, criterion):
    
    global TOTAL_DATA_COUNT, COLUMN_NAMES
    TOTAL_DATA_COUNT = len(train)
    COLUMN_NAMES = train.columns[1:-2]
    
    train = np.array(train)
    
    if criterion == 'mse' or criterion == 'gini':
        root = get_split(train, criterion)
        #print(root)
        split(root, max_depth, min_size, 1, criterion)
        
    elif criterion == 'causal':
        root = get_split(train, criterion)
        split(root, max_depth, min_size, 1, criterion)
    return root

# Functions to use causal tree

In [10]:
#print out the tree structure
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[%s < %.3f]' % ((depth * ' ', (COLUMN_NAMES[node['index'] - 1]), node['value'])))
        print_tree(node['left'], depth + 1)
        print_tree(node['right'], depth + 1)
    else:
        print('%s[%s, %s%%]' % ((depth * ' ', node[0], node[1])))
        
#count the number of leaves in a tree
def count_leaves(node):
    if isinstance(node, tuple) == True:
        return 1
    else:
        return count_leaves(node['left']) + count_leaves(node['right'])
        
        
#causal effect prediction
def causalPredict_helper(node,row):
    if row[node['index'] - 1] < node['value']:
        if isinstance(node['left'], dict):
            return causalPredict_helper(node['left'], row)
        else:
            return node['left'][0]
    else:
        if isinstance(node['right'], dict):
            return causalPredict_helper(node['right'], row)
        else:
            return node['right'][0]    
            
def causalPredict(test, tree):
    #get the information of the trainning set and initialize an empty return dataframe
    column_names = list(test.columns) + ['pred_causal_effect']
    test_matrix = np.array(test)
    ret_matrix = np.empty([0, test_matrix.shape[1] + 1])
    
    #predict for each row
    for row in test_matrix:
        row = np.insert(row, len(row), causalPredict_helper(tree, row))
        ret_matrix = np.append(ret_matrix, [row], axis = 0) 
    
    #return a new dataframe with the predicted value at the end of each row
    ret_df = pd.DataFrame(ret_matrix, columns = column_names)

    return ret_df

# test

### causal tree

In [11]:
#read in data
df = pd.read_csv('test_df.csv')
#df = df.iloc[1:500,:]

#get the column names of predictors
p_str = ['x1', 'x2']
#get the column name of response
r_str = ['y']
#get the column name of treatment
t_str = ['trt']

#set a random seed for replication 
np.random.seed(123)

#split the data
train_set, test_set = causal_train_test_split(df, predictors = p_str, response = r_str, treatment = t_str)
#build a causalTree
tree_test  = causalTree(train_set, max_depth = 3, min_size = 10, criterion = 'causal')

In [12]:
print_tree(tree_test)

[x1 < 6.642]
 [2.873866323777404, 47.1%]
 [x1 < 7.999]
  [-3.189864082840239, 29.2%]
  [-0.980188170731707, 23.7%]


In [13]:
pred_df = causalPredict(test_set, tree_test)
pred_df.head()

,x1,x2,trt,y,pred_causal_effect
0,3.961043,2.619950,0.0,2.12341,2.873866
1,2.999209,2.209014,0.0,1.01134,2.873866
2,7.497546,3.162954,1.0,1.56433,-3.189864
3,10.124939,3.234551,1.0,0.84662,-0.980188
4,3.678320,2.812814,0.0,6.92357,2.873866


# simulation study

## fake data

In [14]:
def fake_response(df):
    
    #design 1
    n_x_1 = 1/2*df['x1'] + df['x2']
    k_x_1 = 1/2*df['x1']
    
    df['design1_y'] = n_x_1 + 1/2*(2*df['treatment'] - 1) * k_x_1 + df['error']
    
    #design 2
    n_x_2 = 1/2 * (df['x1'] + df['x2']) + df['x3'] + df['x4'] + df['x5'] + df['x6']
    
    x1_pos = df['x1'].apply(lambda x: x if x > 0 else 0)
    x2_pos = df['x2'].apply(lambda x: x if x > 0 else 0)
    k_x_2 = x1_pos + x2_pos
    
    df['design2_y'] = n_x_2 + 1/2*(2*df['treatment'] - 1) * k_x_2 + df['error'] 
    
    return df

In [43]:
# create a fake data
fake_data = pd.DataFrame()

np.random.seed(123)
fake_data['x1'] = np.random.normal(0, 0.12, 1000)
fake_data['x2'] = np.random.normal(0.1, 0.2, 1000)
fake_data['x3'] = np.random.gamma(1, 1.5, 1000)
fake_data['x4'] = np.random.beta(1, 1, 1000)
fake_data['x5'] = np.random.logistic(2, 1, 1000)
fake_data['x6'] = np.random.gamma(2, 1, 1000)
fake_data['x7'] = np.random.beta(1, 1.75, 1000)
fake_data['x8'] = np.random.gamma(0.5, 0.5, 1000)
fake_data['x9'] = np.random.beta(0.5, 0.5, 1000)
fake_data['x10'] = np.random.gamma(1.5, 1.5, 1000)
fake_data['treatment'] = np.concatenate((np.zeros(500),np.ones(500)), axis = 0)
fake_data['error'] = np.random.normal(0, 0.01, 1000)

fake_data = fake_response(fake_data)

In [44]:
fake_data.to_csv('fake_data_large.csv')

In [45]:
fake_data.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,treatment,error,design1_y,design2_y
0,-0.130276,-0.049765,0.794089,0.157970,0.830434,0.932308,0.222469,0.201090,0.548776,0.817653,0.0,-0.012080,-0.094414,2.612701
1,0.119681,0.213519,0.689766,0.933238,1.699821,3.747315,0.440415,0.209729,0.963722,7.540478,0.0,-0.013532,0.229907,7.056607
2,0.033957,0.243630,1.756790,0.509386,1.989609,2.818948,0.608559,0.247184,0.442523,4.141402,0.0,-0.001522,0.250598,7.073211
3,-0.180755,-0.099876,2.031662,0.820777,1.183160,0.314951,0.439215,0.034655,0.386692,0.147139,0.0,-0.009023,-0.154088,4.201213
4,-0.069432,0.194980,0.829197,0.989342,0.037687,2.960247,0.550352,0.381596,0.994200,5.076362,0.0,-0.000364,0.177258,4.781393


#### design 1

In [57]:
predictors = ['x1','x2']
response_1 = ['design1_y']
treatment = ['treatment']
for i in range(5):
    print('Tree ' + str(i))
    train_set, test_set = causal_train_test_split(fake_data, predictors, response_1, treatment, test_size = 0, estimation_size = 0.5)
    tree_1  = causalTree(train_set, max_depth = 2, min_size = 25, criterion = 'causal')
    print_tree(tree_1)

Tree 0
[x1 < 0.040]
 [x1 < 0.036]
  [-0.078, 64.1%]
  [-0.089, 1.5%]
 [x2 < 0.143]
  [0.08, 20.1%]
  [0.073, 14.3%]
Tree 1
[x1 < -0.039]
 [x1 < -0.151]
  [-0.206, 11.6%]
  [-0.083, 26.9%]
 [x1 < -0.034]
  [0.34, 1.0%]
  [0.03, 60.5%]
Tree 2
[x1 < -0.047]
 [x1 < -0.061]
  [-0.139, 31.1%]
  [-0.082, 4.2%]
 [x1 < 0.218]
  [0.032, 61.9%]
  [0.061, 2.8%]
Tree 3
[x1 < -0.054]
 [x1 < -0.066]
  [-0.14, 30.0%]
  [-0.168, 2.7%]
 [x2 < 0.061]
  [0.025, 27.3%]
  [0.039, 40.0%]
Tree 4
[x1 < 0.034]
 [x1 < -0.151]
  [-0.21, 11.5%]
  [-0.051, 52.2%]
 [x2 < 0.007]
  [0.079, 12.5%]
  [0.073, 23.8%]


#### design 2

In [ ]:
# predictors = ['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10']
# response_2 = ['design2_y']
# treatment = ['treatment']
# for i in range(5):
#     print('Tree ' + str(i))
#     train_set, test_set = causal_train_test_split(fake_data, predictors, response_2, treatment, test_size = 0, estimation_size = 0.5)
#     tree_2  = causalTree(train_set, max_depth = 5, min_size = 10, criterion = 'causal')
#     print_tree(tree_2)